# Key Rotation

Vault uses unseal keys for the following operations:

- generate root tokens
- seal the Vault
- unseal the Vault

As these operations are highly sensitive, it is very important to keep the keys very secure. One way to ensure the security of these keys, is to rotate them regularly. Moreover, it might be necessary to rotate them due to a leak, or because new key shares are required from a joiner/mover/leaver process. In this tutorial we will learn to work with these keys.

### Starting Vault

Let us start a Vault server. This will run Vault in the background and push the logs to `/tmp/vault.log`. If at any point in time the Vault crashes, this command will need to be used again to re-launch the Vault server.

In [ ]:
nohup bash -c '
  vault server -config ./assets/shamir-config.hcl
' > /tmp/vault.log 2>&1 &
echo $! > /tmp/vault.pid

### Generating a Set of Unseal Keys

We will generate a first set of unseal keys for our Vault. We will use 5 key shares and a threshold of 3.

In [ ]:
export VAULT_ADDR="http://localhost:8200"
vault operator init -format=json | tee /tmp/vault-unseal.json

In [ ]:
# unseal the vault
vault operator unseal "$(jq -r '.unseal_keys_b64[0]' < /tmp/vault-unseal.json)" > /dev/null
vault operator unseal "$(jq -r '.unseal_keys_b64[1]' < /tmp/vault-unseal.json)" > /dev/null
vault operator unseal "$(jq -r '.unseal_keys_b64[2]' < /tmp/vault-unseal.json)" > /dev/null

### Simple Rotations

Simple rotations do not involve any additional layer of security. In other words, the keys are rotated by initialising a rotation, and then providing the current unseal keys to Vault. Once this is done, the new keys are returned. This is typically not ideal, as the person providing the last old key share will see all new keys and need to distribute them. In practice you would want to request Vault to GPG encrypt these key shares first, so that the person entering the last key share gets encrypted data as a response.

Let us start a simple rekey. Update the command below to create 7 key shares (assume the team managing Vault got two new members), with a key threshold still of 3.

In [ ]:
# update the command below to correctly use 7 key shares and a threshold of 3
vault operator rekey -init -key-shares=<x> -key-threshold=<y> -format=json | tee /tmp/vault-rekey.json

In [ ]:
# check the status of the current rekey operation
vault operator rekey -status

In [ ]:
# provide first unseal key share
vault operator rekey -nonce="$(jq -r '.nonce' < /tmp/vault-rekey.json)" "$(jq -r '.unseal_keys_b64[0]' < /tmp/vault-unseal.json)"

In [ ]:
# provide second unseal key share
vault operator rekey -nonce="$(jq -r '.nonce' < /tmp/vault-rekey.json)" "$(jq -r '.unseal_keys_b64[1]' < /tmp/vault-unseal.json)" > /dev/null

In [ ]:
# provide third and last unseal key share
vault operator rekey -format=json -nonce="$(jq -r '.nonce' < /tmp/vault-rekey.json)" "$(jq -r '.unseal_keys_b64[2]' < /tmp/vault-unseal.json)" | tee /tmp/vault-rekey-result.json

Nice, once you have done this, you completed a rotation. At this point, the key shares provided in the output would need to be distributed to the team members. This is where the issues can start, as the person distributing the keys obviously has access to these keys, and therefore needs to be trusted to not store them somewhere for later use. A better process is to ensure these are encrypted and can only be seen by the person the key share is meant to be.

### GPG Encrypted Key Shares

When using GPG, every user needs to have a GPG keys set. This consists of a public key and a private key. In this scenario we will use the public key to encrypt the key shares directly in Vault. Thus only the people in posession of the private key can decrypt and thus use their key share. Here we will create 3 GPG key pairs. Of course, in the real world, every person would only create one such key pair and share the public key with others either via a key server or via trusted channels. For the sake of simplicity we are creating all three key pair locally here.

In [ ]:
mkdir .gnupg
export GNUPGHOME="$(pwd)/.gnupg"
cat > /tmp/user1 <<EOF
    %echo Generating a basic OpenPGP key
    Key-Type: RSA
    Key-Length: 2048
    Subkey-Type: RSA
    Subkey-Length: 2048
    Name-Real: User 1
    Name-Comment: User 1
    Name-Email: user1@example.com
    Expire-Date: 0
    %no-ask-passphrase
    %no-protection
    %commit
    %echo done
EOF
gpg --batch --gen-key /tmp/user1
cat > /tmp/user2 <<EOF
    %echo Generating a basic OpenPGP key
    Key-Type: RSA
    Key-Length: 2048
    Subkey-Type: RSA
    Subkey-Length: 2048
    Name-Real: User 2
    Name-Comment: User 2
    Name-Email: user2@example.com
    Expire-Date: 0
    %no-ask-passphrase
    %no-protection
    %commit
    %echo done
EOF
gpg --batch --gen-key /tmp/user2
cat > /tmp/user3 <<EOF
    %echo Generating a basic OpenPGP key
    Key-Type: RSA
    Key-Length: 2048
    Subkey-Type: RSA
    Subkey-Length: 2048
    Name-Real: User 3
    Name-Comment: User 3
    Name-Email: user3@example.com
    Expire-Date: 0
    %no-ask-passphrase
    %no-protection
    %commit
    %echo done
EOF
gpg --batch --gen-key /tmp/user3

In [ ]:
# List the keys and validate that they were created correctly.
gpg --list-keys

### Exporting Public Keys

Once the key pairs are generated, the public keys can either be shared via a key server, or provided to Vault as files. Here we will use the files approach. Note that this is actually quite common. For instance, if you already have a secure version control system set up, and sign you commits, you can easily share your public keys on your version control system by commiting them somewhere and signing that commit. That way people can verify that the public key truly did originate from you and trust its integrity. Let us export the public keys to local files:

In [ ]:
gpg --export -a user1@example.com > user1-pub.asc
gpg --export -a user2@example.com > user2-pub.asc
gpg --export -a user3@example.com > user3-pub.asc

Now a rekey can be triggered using the `-pgp-keys` flag, where the list of files can be provided. Note that the list of files must have the same length as the keys shares that are being generated. Initialise the key rotation:

In [ ]:
vault operator rekey -init -key-shares=<x> -key-threshold=<y> -pgp-keys="<comma-list-of-files>" -format=json | tee /tmp/vault-rekey-gpg.json

### Verifying The Rekey

Now, as a Vault operator, I would want to know that whoever initiated the key rotation truly provided the correct GPG keys before I submit my Vault key share to push the progress of the rekey. This can be done via the rekey status command we used before. This will show the GPG public key IDs that were submitted. I can then compare these IDs to my known public keys, or submit them to a key server to figure out who they belong to.

For instance, you can search for my public key ID `96A8BA6EC8712183` on https://keys.openpgp.org/. If you download the public key you will see that it is linked to my GitHub email address `f4z3r-github@pm.me`. You can also validate this by searching by the email address on https://keys.openpgp.org/ and compare the key signature it finds with the one you found by searching by ID.

So now you can execute that command and check the IDs. These should match the IDs that were shown by the `gpg --list-keys` command above.

In [ ]:
vault operator rekey -status

Now that this is validated, let us trigger the rekey by providing 3 key shares:

In [ ]:
vault operator rekey -nonce="$(jq -r '.nonce' < /tmp/vault-rekey-gpg.json)" "$(jq -r '.unseal_keys_b64[0]' < /tmp/vault-rekey.json)"
vault operator rekey -nonce="$(jq -r '.nonce' < /tmp/vault-rekey-gpg.json)" "$(jq -r '.unseal_keys_b64[1]' < /tmp/vault-rekeyw.json)"
vault operator rekey -format=json -nonce="$(jq -r '.nonce' < /tmp/vault-rekey-gpg.json)" "$(jq -r '.unseal_keys_b64[2]' < /tmp/vault-rekey.json)" | tee /tmp/vault-rekey-gpg-result.json

### Decrypting a Key Share

As you can see, the key shares are much longer now. This is because they are encrypted by GPG. You can have a look at what their representation by base64 decoding them. But much more interestingly we might want to decrypt such a key share. In practice, the person that performs the last rekey would send the encrypted key shares to the people owning the public keys. He/she cannot do anything with the encrypted key shares.

When someone receives a key share for which they own the private GPG key, then can then decrypt the key share as done below. Of course, since we are now the owner of all three GPG private keys, we can decrypt all three. In practice, we would only have access to our own GPG private key.

In [ ]:
jq -r '.keys_base64[0]' < /tmp/vault-rekey-gpg-result.json | base64 -d | gpg --decrypt

## Cleaning Up

At the end of each module, you should clean up your Vault instance. This is done by shutting it down and wiping its database to restore its state.

In [ ]:
kill $(cat /tmp/vault.pid)
rm /tmp/vault/vault.db
rm -r /tmp/vault/raft/
rm /tmp/vault.log
rm /tmp/vault.pid
rm /tmp/vault-unseal.json
rm /tmp/vault-rekey.json
rm /tmp/vault-rekey-result.json
rm -rf .gnupg/
rm *.asc